Copyright **`(c)`** 2025 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
from itertools import combinations
import numpy as np

## Simple Test Problem

In [2]:
CITIES = [
    "Rome",
    "Milan",
    "Naples",
    "Turin",
    "Palermo",
    "Genoa",
    "Bologna",
    "Florence",
    "Bari",
    "Catania",
    "Venice",
    "Verona",
    "Messina",
    "Padua",
    "Trieste",
    "Taranto",
    "Brescia",
    "Prato",
    "Parma",
    "Modena",
]
test_problem = np.load('lab2/test_problem.npy')

FileNotFoundError: [Errno 2] No such file or directory: 'lab2/test_problem.npy'

## Common tests

In [8]:
problem = np.load('lab2/problem_r2_100.npy')

In [9]:
# Negative values?
np.any(problem < 0)

np.True_

In [10]:
# Diagonal is all zero?
np.allclose(np.diag(problem), 0.0)

False

In [11]:
# Symmetric matrix?
np.allclose(problem, problem.T)

False

In [12]:
# Triangular inequality
all(
    problem[x, y] <= problem[x, z] + problem[z, y]
    for x, y, z in list(combinations(range(problem.shape[0]), 3))
)

False